In [ ]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


train_images=np.expand_dims(train_images,axis=3)
train_images = train_images / 255.0

test_images=np.expand_dims(test_images,axis=3)
test_images = test_images / 255.0

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

In [ ]:
IMAGE_SHAPE = [28, 28, 1]

inputs = tf.keras.Input(shape=(*IMAGE_SHAPE,))
x = tf.keras.layers.Conv2D(32, kernel_size=5, padding='SAME', activation=tf.nn.relu)(inputs)
x = tf.keras.layers.MaxPooling2D(
    pool_size=[2, 2], strides=[2, 2], padding="SAME")(x)
x = tf.keras.layers.Conv2D(64, kernel_size=5, padding='SAME', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D(
    pool_size=[2, 2], strides=[2, 2], padding="SAME")(x)
x = tf.keras.layers.Conv2D(128, kernel_size=5, padding='SAME', activation=tf.nn.relu)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.layers.Dense(
    84, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.4)(x)
predictions = tf.layers.Dense(10,activation=tf.nn.softmax)(x)

cnn = tf.keras.Model(inputs=inputs, outputs=predictions)


checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

cnn.compile(optimizer='adam', 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
Train_CNN=False

if Train_CNN:
    cnn.fit(train_images, train_labels, epochs=200,
          callbacks = [cp_callback])
else:
    cnn.load_weights(checkpoint_path)

In [ ]:
test_loss, test_acc = cnn.evaluate(test_images, test_labels)



print('Test accuracy:', test_acc)

# Using layer as other model's output

In [ ]:
cnn.summary()

In [ ]:

class distiller(tf.keras.Model):
    
    def __init__(self, ext_model,lay_num, hidden_size=100):

        super(distiller, self).__init__()
        self.IMAGE_SHAPE  = [28, 28, 1]

        self.FILTER_SHAPE = ext_model.layers[lay_num].output.shape.as_list()[1:]

        self.ext_model_layers=ext_model.layers[:lay_num]
        for lay in self.ext_model_layers:
            lay.trainable = False
        
        self.ext_post_net=ext_model.layers[lay_num:]
        for lay in self.ext_post_net:
            lay.trainable = False
        
        #for layer in intermediate_layer_model.layers:
        #    layer.trainable = False


        self.rs = tf.keras.layers.Reshape([-1,np.prod(self.IMAGE_SHAPE)])
        self.hidden = tf.keras.layers.Dense(hidden_size, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(np.prod(self.FILTER_SHAPE), activation=tf.nn.relu)
        #self.dropout = tf.keras.layers.Dropout(0.5)
    


    def call(self, inputs, training=False):

        y=self.out(self.hidden(self.rs(inputs)))

        if training:
            y_ext = inputs
            for lay in self.ext_model_layers:
                y_ext = lay(y_ext)
            y_ext.reshape([-1,y_ext.output.shape.as_list()])
            return tf.math.subtract(t,y_ext)

        return y
    
    def mix_pred(self,inputs):
        
        y=self.out(self.hidden(self.rs(inputs)))
        y.reshape([-1,self.FILTER_SHAPE])
        for lay in self.ext_post_net:
                y = lay(y)
        return y



def loss(y_true,y_pred):
    return tf.mean_square_error(un[0], tf.zeros_like(y_true))




In [ ]:
sizes = [10,100,500,1000,10000]

In [ ]:
def run(hidden_size):
    global cnn
    model = distiller(cnn, 5, hidden_size=hidden_size)
    model.compile(loss='mean_squared_error',
                  metrics=['mean_squared_error'],
                 optimizer=keras.optimizers.Adam())
    checkpoint_path = "training_distiller"+str(hidden_size)+"/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                     save_weights_only=True,
                                                     verbose=1)

    shape_out=np.prod(cnn.layers[6].output.shape.as_list()[1:])
    model.fit(train_images, 
              np.zeros([train_images.shape[0],shape_out]), 
              epochs=50,
              callbacks = [cp_callback])

    np.save("c5u"+str(hidden_size),model.history.history['loss'])

In [ ]:
for s in sizes:
    run(s)

# Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
for i in sizes:
    plt.semilogy(np.load("c5u"+str(i)+".npy"), label=str(i))
plt.legend()
plt.savefig("losses.png")